In [1]:
# instalação do pyspark
%pip install pyspark==3.3.1

     |████████████████████████████████| 281.4 MB 8.1 kB/s eta 0:00:01    |████████▋                       | 75.4 MB 6.9 MB/s eta 0:00:31     |███████████▋                    | 101.8 MB 4.5 MB/s eta 0:00:40     |██████████████████              | 159.1 MB 6.6 MB/s eta 0:00:19     |██████████████████████          | 194.0 MB 6.6 MB/s eta 0:00:14     |██████████████████████▉         | 201.1 MB 3.0 MB/s eta 0:00:27     |████████████████████████        | 210.3 MB 5.9 MB/s eta 0:00:13     |█████████████████████████▏      | 221.0 MB 6.9 MB/s eta 0:00:09     |██████████████████████████▋     | 234.1 MB 3.7 MB/s eta 0:00:13     |███████████████████████████     | 237.8 MB 4.0 MB/s eta 0:00:11     |███████████████████████████▌    | 241.7 MB 2.7 MB/s eta 0:00:15     |███████████████████████████▉    | 245.0 MB 2.8 MB/s eta 0:00:13     |████████████████████████████▌   | 250.7 MB 4.2 MB/s eta 0:00:08     |████████████████████████████▊   | 252.4 MB 4.2 MB/s eta 0:00:07     |█████████████████████████████▋

In [1]:
# importando classe SparkSession ponto de entrada ler dados, consultas SQL
from pyspark.sql import SparkSession

In [2]:
# criando uma instância da classe 'SparkSession' no ambiente PySpark
# iniciando a construção da sessão Spark, definindo nome da aplicação
# e retornando a SparkSession existente ou cria uma nova se não existir
spark = SparkSession\
    .builder\
    .appName('api_transformation')\
    .getOrCreate()

24/01/12 15:20:35 WARN Utils: Your hostname, alex-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.112.128 instead (on interface ens33)
24/01/12 15:20:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/01/12 15:20:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Definindo df de em um arquivo JSON
df = spark.read.json('../../datalake/api_datascience')

In [4]:
# spark exibindo as primeiras linhas do DataFrame
df.show()

+--------------------+--------------------+--------------------+------------------+------------+
|     _corrupt_record|                data|            includes|              meta|extract_date|
+--------------------+--------------------+--------------------+------------------+------------+
|                null|[{74, 16, 2024-01...|{[{2024-01-07T06:...|{1234567890abcdef}|  2024-01-07|
|                null|[{14, 77, 2024-01...|{[{2024-01-07T16:...|{1234567890abcdef}|  2024-01-07|
|                null|[{43, 79, 2024-01...|{[{2024-01-07T19:...|              null|  2024-01-07|
|                null|[{40, 73, 2024-01...|{[{2024-01-05T21:...|{1234567890abcdef}|  2024-01-05|
|                null|[{44, 14, 2024-01...|{[{2024-01-05T00:...|              null|  2024-01-05|
|                null|[{16, 40, 2024-01...|{[{2024-01-06T17:...|              null|  2024-01-06|
|                null|[{37, 44, 2024-01...|{[{2024-01-09T23:...|              null|  2024-01-09|
|                null|[{57, 70

In [5]:
# analisando estrutura do esquema do dataframe
df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: str

Extraindo informações de forma simplificada em colunas

In [6]:
# importando módulo que permite realizar operações em colunas DataFrames Spark
from pyspark.sql import functions as f

In [7]:
# transformando data array em linhas e imprimindo a nova estrutura em um formato mais plano para análise
df.select(f.explode('data')).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [8]:
# retornando os 5 primeiros registros
df.select(f.explode('data')).show(5)

+--------------------+
|                 col|
+--------------------+
|{74, 16, 2024-01-...|
|{89, 94, 2024-01-...|
|{71, 64, 2024-01-...|
|{74, 66, 2024-01-...|
|{35, 32, 2024-01-...|
+--------------------+
only showing top 5 rows



In [9]:
# apelidadando a coluna col no dataframe de 'tweets'
# pegando cada informação em uma coluna através do segundo select acessando os dados de interesse
# utilizando recurso do PySpark '.*' para acessar todos os campos q pertecem à metrics
df.select(f.explode("data").alias("tweets"))\
.select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [10]:
# salvando novo dataframe em uma variável
tweet_df = df.select(f.explode("data").alias("tweets"))\
        .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [11]:
tweet_df.show(5)

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       74|             16|2024-01-07T19:19:...| 51|        99|         31|         40|           31|Um terceiro tweet...|
|       89|             94|2024-01-07T21:24:...| 83|        49|         76|         92|           16|Outro tweet fictí...|
|       71|             64|2024-01-07T12:32:...| 40|         4|         58|         90|            1|Tweet fictício cr...|
|       74|             66|2024-01-07T12:55:...| 18|        47|         10|          5|           64|Este é um tweet f...|
|       35|             32|2024-01-07T05:34:...| 44|         4|         27|         46|           14|Este é um tweet f...|
+---------+-----

In [12]:
# transformando users array em linhas e imprimindo a nova estrutura em um formato mais plano para análise
df.select(f.explode('includes.users')).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- username: string (nullable = true)



In [13]:
# criando um alias de nome coluna 'col' para a novo nome coluna 'users'
df.select(f.explode('includes.users').alias('users'))

DataFrame[users: struct<created_at:string,id:string,name:string,username:string>]

In [14]:
# selecionando todos os campos de informações dos usuários
# transformando os dados em colunas e para primeiro nível do schema
df.select(f.explode('includes.users').alias('users')).select('users.*').printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [15]:
# salvando dataframe atualizado em uma nova variável
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [16]:
# visualizando dataframe atualizado
user_df.show(5)

+--------------------+---+------+--------+
|          created_at| id|  name|username|
+--------------------+---+------+--------+
|2024-01-07T06:25:...| 35|User 1|   user1|
|2024-01-07T17:43:...| 47|User 2|   user2|
|2024-01-07T22:37:...| 38|User 3|   user3|
|2024-01-07T17:33:...| 88|User 4|   user4|
|2024-01-07T02:22:...| 75|User 5|   user5|
+--------------------+---+------+--------+
only showing top 5 rows



In [17]:
# unindo as informações do Spark para salvar em um único arquivo
# salvando as informações em um arquivo JSON
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
user_df.coalesce(1).write.mode("overwrite").json('output/user')